# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import re
import pickle
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import warnings
warnings.filterwarnings("ignore")

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df.message.values
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
# Write tokenization function to process text data
def tokenize(text):
    
    # Normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())

    # Token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]

    # Lemmatizer and clean
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Build machine learning pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Evaluate metrics of ML pipeline 
def eval_metrics(ArrayL, ArrayP, col_names):
    
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics

In [9]:
# Calculate evaluation metrics for training set
y_train_pred = pipeline.predict(X_train)
col_names = list(y.columns.values)

print(eval_metrics(np.array(y_train), y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.989601   0.990452  0.995972  0.993204
request                 0.986527   0.997428  0.924062  0.959345
offer                   0.998873   1.000000  0.750000  0.857143
aid_related             0.984068   0.993970  0.967829  0.980725
medical_help            0.986732   0.999224  0.833118  0.908642
medical_products        0.992879   0.998817  0.859470  0.923919
search_and_rescue       0.994570   0.997738  0.807692  0.892713
security                0.995594   1.000000  0.759104  0.863057
military                0.995338   1.000000  0.861702  0.925714
water                   0.994672   0.999136  0.918254  0.956989
food                    0.995236   0.997633  0.959909  0.978407
shelter                 0.992111   0.999365  0.911407  0.953362
clothing                0.998207   1.000000  0.882943  0.937833
money                   0.995851   1.000000  0.823913  0.903456
missing_people          0.996978   0.994

In [10]:
# Calculate evaluation metrics for test set
y_test_pred = pipeline.predict(X_test)

eval_metrics0 = eval_metrics(np.array(y_test), y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.818042   0.851322  0.925379  0.886807
request                 0.882434   0.781701  0.436380  0.560092
offer                   0.995390   0.000000  0.000000  0.000000
aid_related             0.755494   0.742035  0.624581  0.678261
medical_help            0.920240   0.637681  0.081784  0.144975
medical_products        0.950515   0.600000  0.081571  0.143617
search_and_rescue       0.973260   0.583333  0.078652  0.138614
security                0.982173   0.000000  0.000000  0.000000
military                0.969110   0.523810  0.054455  0.098655
water                   0.956047   0.835106  0.381068  0.523333
food                    0.932995   0.816017  0.517857  0.633613
shelter                 0.929307   0.792627  0.293015  0.427861
clothing                0.984939   0.681818  0.141509  0.234375
money                   0.978177   0.666667  0.027778  0.053333
missing_people          0.989550   0.333

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Review params to get ideas to improve model
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f367844a620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
# Perform grid search to find better parameters
parameters = {'tfidf__use_idf': (True, False),
              'vect__ngram_range': ((1, 1), (1, 2)),
              'clf__estimator__n_estimators': [10,20],
}
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3)

# Find best parameters
T_model = cv.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.2338899702899293, total= 1.8min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.22961065573770492, total= 1.8min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.0min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.24024177850630057, total= 2.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.23278688524590163, total= 2.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.23552914660383156, total= 1.8min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.22899590163934427, total= 1.8min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 2), score=0.23911484479049278, total= 2.3min
[CV] clf__estimator

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 49.9min finished


In [13]:
# Grid search results
T_model.cv_results_

{'mean_fit_time': array([  63.16922081,   86.16967964,   63.80121577,   88.35545719,
          84.76486337,  131.52965391,   86.42719221,  139.10420871]),
 'std_fit_time': array([ 1.23045409,  2.87658978,  1.4163779 ,  3.41441476,  2.00575626,
         5.84891903,  2.03989244,  1.78708339]),
 'mean_score_time': array([ 43.9661237 ,  45.47683251,  44.20432281,  45.65678787,
         45.97858405,  48.80975175,  46.11071301,  54.59068859]),
 'std_score_time': array([ 0.75211608,  0.82379854,  1.07510853,  0.7555356 ,  0.73018622,
         0.53480983,  0.72210574,  6.48409736]),
 'param_clf__estimator__n_estimators': masked_array(data = [10 10 10 10 20 20 20 20],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False False True True False False],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__ngram_range': masked_array(data 

In [14]:
# Top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.25111418472414321

In [15]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Evaluate metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.814200   0.853242  0.916401  0.883694
request                 0.884278   0.766520  0.467742  0.580968
offer                   0.995390   0.000000  0.000000  0.000000
aid_related             0.757799   0.751246  0.617505  0.677841
medical_help            0.918857   0.578125  0.068773  0.122924
medical_products        0.953127   0.782609  0.108761  0.190981
search_and_rescue       0.973413   0.647059  0.061798  0.112821
security                0.982173   0.000000  0.000000  0.000000
military                0.968495   0.428571  0.044554  0.080717
water                   0.959736   0.867647  0.429612  0.574675
food                    0.937452   0.842217  0.542582  0.659983
shelter                 0.929153   0.800000  0.286201  0.421581
clothing                0.984017   0.600000  0.056604  0.103448
money                   0.978331   0.800000  0.027778  0.053691
missing_people          0.988935   0.000

In [17]:
# Summary of first model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943511,0.539841,0.196419,0.251829
std,0.055402,0.307257,0.247707,0.271017
min,0.755494,0.000000,0.000000,0.000000
25%,0.934455,0.375000,0.021352,0.040671
50%,0.956047,0.666667,0.078652,0.138614
75%,0.980867,0.757401,0.345598,0.482270
max,0.996465,0.896714,0.925379,0.886807


In [18]:
# Summary of tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943735,0.545996,0.187520,0.239194
std,0.055492,0.336340,0.251418,0.278861
min,0.757799,0.000000,0.000000,0.000000
25%,0.934609,0.339286,0.004805,0.009485
50%,0.959736,0.647059,0.056291,0.103448
75%,0.980713,0.800000,0.343119,0.480979
max,0.996465,1.000000,0.916401,0.883694


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# Improve the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [20]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline 2
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [21]:
# Calculate evaluation metrics for training set 2
y_train_pred2 = pipeline2.predict(X_train)
col_names_ = list(y.columns.values)

print(eval_metrics(np.array(y_train), y_train_pred2, col_names_))

                        Accuracy  Precision    Recall        F1
related                 0.765842   0.765967  0.999398  0.867249
request                 0.828390   1.000000  0.000895  0.001788
offer                   0.995185   1.000000  0.010526  0.020833
aid_related             0.586394   0.518904  0.116296  0.190008
medical_help            0.920240   1.000000  0.001283  0.002562
medical_products        0.949490   1.000000  0.002024  0.004040
search_and_rescue       0.971825   1.000000  0.007220  0.014337
security                0.982275   1.000000  0.002882  0.005747
military                0.966856   1.000000  0.003082  0.006144
water                   0.936069   1.000000  0.001600  0.003195
food                    0.888069   1.000000  0.000457  0.000914
shelter                 0.911377   1.000000  0.002882  0.005747
clothing                0.984888   1.000000  0.006734  0.013378
money                   0.976538   1.000000  0.004348  0.008658
missing_people          0.988371   1.000

In [22]:
# Calculate evaluation metrics for test set 2
y_test_pred2 = pipeline2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(y_test), y_test_pred2, col_names)
print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.764100   0.764289  0.998993  0.866021
request                 0.827724   0.000000  0.000000  0.000000
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.581374   0.494436  0.114464  0.185894
medical_help            0.919318   0.000000  0.000000  0.000000
medical_products        0.950054   0.000000  0.000000  0.000000
search_and_rescue       0.973874   0.000000  0.000000  0.000000
security                0.980944   0.000000  0.000000  0.000000
military                0.967420   0.000000  0.000000  0.000000
water                   0.935147   0.000000  0.000000  0.000000
food                    0.886737   0.000000  0.000000  0.000000
shelter                 0.910250   0.000000  0.000000  0.000000
clothing                0.983249   0.000000  0.000000  0.000000
money                   0.977716   0.000000  0.000000  0.000000
missing_people          0.989089   0.000

In [23]:
# Look at parameters for improving model
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('best', TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
          random_state=None, tol=0.0)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
    

In [24]:
# Model tuning with added parameters
parameters2 = {'tfidf__use_idf': (True, False), 
               'clf__estimator__n_estimators': [50, 100],
               'clf__estimator__learning_rate': [1,2]
              }
cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters2, cv=2, n_jobs=-1, verbose=3)

# Find best parameters
T_model2 = cv2.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.17723593894068232, total=  20.2s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   23.3s remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.19088114754098362, total=  20.3s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   46.7s remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.17682614486220674, total=  20.2s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.18770491803278688, total=  20.3s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.16678618993955538, total=  39.8s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, score=0.18575819672131147, total=  39.5s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=100, tfidf__use_idf

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 12.7min finished


In [25]:
# Top mean score for model 2
np.max(T_model2.cv_results_['mean_test_score'])

0.18405819374007482

In [26]:
# Parameters for top mean score model 2
T_model2.best_params_

{'clf__estimator__learning_rate': 1,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True}

### Testing Model

In [27]:
# Evaluating metrics for test set model 2
tuned_pred_test2 = T_model2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.764100   0.764289  0.998993  0.866021
request                 0.827724   0.000000  0.000000  0.000000
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.581374   0.494436  0.114464  0.185894
medical_help            0.919318   0.000000  0.000000  0.000000
medical_products        0.950054   0.000000  0.000000  0.000000
search_and_rescue       0.973874   0.000000  0.000000  0.000000
security                0.980944   0.000000  0.000000  0.000000
military                0.967266   0.000000  0.000000  0.000000
water                   0.935147   0.000000  0.000000  0.000000
food                    0.886737   0.000000  0.000000  0.000000
shelter                 0.910250   0.000000  0.000000  0.000000
clothing                0.983249   0.000000  0.000000  0.000000
money                   0.977716   0.000000  0.000000  0.000000
missing_people          0.989089   0.000

In [28]:
# Summary of original model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943511,0.539841,0.196419,0.251829
std,0.055402,0.307257,0.247707,0.271017
min,0.755494,0.000000,0.000000,0.000000
25%,0.934455,0.375000,0.021352,0.040671
50%,0.956047,0.666667,0.078652,0.138614
75%,0.980867,0.757401,0.345598,0.482270
max,0.996465,0.896714,0.925379,0.886807


In [29]:
# Summary of tuned model 1
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943735,0.545996,0.187520,0.239194
std,0.055492,0.336340,0.251418,0.278861
min,0.757799,0.000000,0.000000,0.000000
25%,0.934609,0.339286,0.004805,0.009485
50%,0.959736,0.647059,0.056291,0.103448
75%,0.980713,0.800000,0.343119,0.480979
max,0.996465,1.000000,0.916401,0.883694


In [30]:
# Summary of tuned model 2
eval_metrics2.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.923919,0.101916,0.032208,0.030836
std,0.089345,0.247148,0.169329,0.148676
min,0.581374,0.000000,0.000000,0.000000
25%,0.911019,0.000000,0.000000,0.000000
50%,0.953588,0.000000,0.000000,0.000000
75%,0.981020,0.000000,0.000000,0.000000
max,0.996465,1.000000,0.998993,0.866021


### Evaluating Tuned Models

* Model 1 has higher mean in all categories (accuracy, precision, recall, F1 value). The standard deviation for model 1 is higher, indicating a greater distribution in relation to the mean values
* Model 2 has lower mean value for all categories and smaller standard deviation for each metric when compared to model 1

* Comparing metrics (mean value) for model 1 and model 2
>* **Accuracy**: Both model 1 and model 2 have pretty high accuracy (0.942185 and 0.925368, respectively)
>* **Precision**: Model 1 has a higher average precision of 0.559671 when compared to model 2 (0.081468). The higher precision of model 1, indicates that this model is better at predicting actual positives and is better for predicting when the costs of a false positive is high.
>* **Recall**: Model 1 has a higer recall of 0.171154 when compared to model 2's recall of 0.030304. The higher recall value in model 1, calculates how many of the actual positives the model captures that are a true positive. This means model 1 would be best selected when there's a high cost associated with false negatives. 
>* **F1**: Model 1 has a higher F1 value of 0.225254 when compared to model 2 (0.028043). The F1 score is best used if we're seeking to find a balance between Precision and Recall and if there's an uneven distribution. An F1 score of 1 is considered a 'perfect' model, while and F1 score of 0 indicates a bad model. The higher F1 value for model 1 is indicative that the model has low false positives and low false negatives, so it's more effective at correctly identifying real threats and isn't disturbed by false alarms. 

* Based on the metrics, Model 1 is the best overall model and will be saved for continuation of this work

### 9. Export your model as a pickle file

In [31]:
# Export best model as pickle file
pickle.dump(T_model, open('classifer.pkl', 'wb'))

### Sources
* https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
* https://pathmind.com/wiki/accuracy-precision-recall-f1#:~:text=That%20is%2C%20a%20good%20F1,total%20failure%20when%20it's%200%20
* https://apapiu.github.io/2016-08-04-tf_idf/
* https://www.greenbook.org/marketing-research/how-to-interpret-standard-deviation-and-standard-error-in-survey-research-03377
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html